In [16]:
from pyquery import PyQuery
from selenium import webdriver
from selenium.webdriver.common.desired_capabilities import DesiredCapabilities
from selenium.common.exceptions import TimeoutException, NoSuchElementException
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver import FirefoxProfile
from selenium.webdriver.support.ui import Select
import os
import time
import csv

def record_csv(headers, content, path):
  with open(path, 'w', encoding='utf-8') as f:
    f_csv = csv.writer(f, lineterminator='\n')
    f_csv.writerow(headers)
    f_csv.writerows(content)
  print('save csv done', path)


In [17]:

class DriverFindElementByCSS:
  ''' usage:
      driver / css(pat)  => one element
      driver // css(pat)  => [element list]
  '''
  def __call__(self, pat):
    self.pat = pat
    return self
  def __rtruediv__(self, driver):
    return driver.find_element_by_css_selector(self.pat)
  def __rfloordiv__(self, driver):
    return driver.find_elements_by_css_selector(self.pat)

css = DriverFindElementByCSS()


In [36]:
gecko_path = '/Users/probe/git/Lottery_Hedge_Crawler/geckodriver'
gecko_path = 'C:/Program Files/Mozilla Firefox/geckodriver.exe'
profile = webdriver.FirefoxProfile()
# profile.set_preference("browser.download.dir","/Users/probe/git/StateCancerProfilesCrawler/");
# profile.setPreference("browser.download.folderList", 2);
profile.set_preference("browser.helperApps.neverAsk.saveToDisk", "application/octet-stream,text/csv");
profile.set_preference("browser.download.dir",os.getcwd())

# if proxies:
#   profile.set_preference("network.proxy.type", 1)
#   profile.set_preference("network.proxy.socks", "127.0.0.1")
#   profile.set_preference("network.proxy.socks_port", 1080)
#   profile.set_preference("network.proxy.socks_version", 5)
#   # profile.setAssumeUntrustedCertificateIssuer(False)
#   profile.update_preferences()
driver = webdriver.Firefox(firefox_profile=profile,
                           executable_path=gecko_path)


In [34]:

def fetch_one_page(state_code):
  driver.get('https://ofmpub.epa.gov/apex/sfdw/f?p=108:200::::::')
  print('get map page')
#   state_code = '09'
  selector = Select(driver / css('#P200_STATE'))
  selector.select_by_value(state_code)
  search_button = driver / css('#P200_SEARCH')
  search_button.click()
  print('search')
  WebDriverWait(driver, 120).until(
      EC.presence_of_element_located((By.CSS_SELECTOR, '#SFDWIR_row_select'))
  )
  change_all_selector = Select(driver / css('#SFDWIR_row_select'))
  change_all_selector.select_by_value('100000')
  print('click all rows')
  WebDriverWait(driver, 120).until(
      EC.invisibility_of_element_located((By.CSS_SELECTOR, 'span.u-Processing-spinner'))
  )
  WebDriverWait(driver, 400).until(
      EC.presence_of_element_located((By.CSS_SELECTOR, '.t-fht-tbody'))
  )

  time.sleep(10)
  print('prepare to fetch')
  tables_header, tables_body = driver // css('.a-IRR-table')
  headers = [td.text.replace('\n', ' ') for td in tables_header // css('th')]

  
  content = []
  for row in PyQuery(tables_body.get_attribute('outerHTML'))('tr'):
    row = PyQuery(row)
    if not row:
      continue
    row = [td.text for td in row('td')]
    if row and row[0]:
      content.append(row)
  record_csv(headers, content, 'epa_water_system_summary_{}_[{}].csv'.format(state_code, len(content)))
  

In [41]:
# why DC connot fetch

# 03
codes = '''

04
05
06
07
08
09
10
NN
'''


for state_code in codes.strip().splitlines():
  fetch_one_page(state_code=state_code)

get map page
search
click all rows
prepare to fetch
save csv done epa_water_system_summary_04_[120].csv
get map page
search
click all rows
prepare to fetch
save csv done epa_water_system_summary_05_[282].csv
get map page
search
click all rows
prepare to fetch
save csv done epa_water_system_summary_06_[151].csv
get map page
search
click all rows
prepare to fetch
save csv done epa_water_system_summary_07_[21].csv
get map page
search
click all rows
prepare to fetch
save csv done epa_water_system_summary_08_[239].csv
get map page
search
click all rows
prepare to fetch
save csv done epa_water_system_summary_09_[567].csv
get map page
search
click all rows
prepare to fetch
save csv done epa_water_system_summary_10_[202].csv
get map page
search
click all rows
prepare to fetch
save csv done epa_water_system_summary_NN_[250].csv
